# Integration test to check that all latest modules run

In [ ]:
import json
import os
import sys
import tarfile
from glob import glob
import re

from collections import Counter
import requests
from datetime import datetime
from pathlib import Path

import tengu

### 0) Setup

In [ ]:
# Set our token - ensure you have exported TENGU_TOKEN in your shell; or just replace the os.getenv with your token
TOKEN = os.getenv("TENGU_TOKEN")
URL = os.getenv("TENGU_URL")

In [ ]:
# Get our client, for calling modules and using the tengu API
client = tengu.Provider(access_token=TOKEN, url=URL)

In [ ]:
modules = client.get_latest_module_paths()

In [ ]:
modules

{'hermes_binding': 'github:talo/tengu-prelude/083e958d64e9e716754f76560bd3e60557e6a691#hermes_binding',
 'gmx_frame_select_pdb': 'github:talo/gmx_tengu_support/68cfc9dc1533bd54dfa21b346642410ff0fce570#gmx_frame_select_pdb',
 'gmx_frame_select': 'github:talo/gmx_tengu_support/a15ebf3e86b779203b463193c0ecc025c9f916df#gmx_frame_select',
 'gmx_mmpbsa_tengu': 'github:talo/gmx_tengu_support/d4ea797dcdedf9b91de1b76a32f8a95f0cbf21df#gmx_mmpbsa_tengu',
 'tengu_echo': 'github:talo/tengu-module-example/b334a851530033b79762c3341bf584f8939feee1#tengu_echo',
 'spam': 'github:talo/tengu-module-example/b334a851530033b79762c3341bf584f8939feee1#spam',
 'delay': 'github:talo/tengu-module-example/b334a851530033b79762c3341bf584f8939feee1#delay',
 'dummy': 'github:talo/tengu-module-flake-parts/b6afd5abb624fed203990045927d381cefc6ef04#dummy',
 'gmx_tengu_pdb': 'github:talo/gmx_tengu_support/a473bc4a302eebebcb5f54a899192be75c0daa91#gmx_tengu_pdb',
 'gmx_tengu': 'github:talo/gmx_tengu_support/a473bc4a302eebebc

In [ ]:
deploy_instances = [next(client.module_instances(tags=["deploy"], path=path, status="COMPLETED")) for path in modules.values()]

In [ ]:
for instance_set in deploy_instances:
    for instance in instance_set:
        if not "retry" in str.join("", instance["tags"]):
            retry = client.retry(instance["id"])
            print("retrying", instance["id"], "as", retry)

retrying 3ac69002-d2b8-41ea-a3bc-1d1eef2d3d8f as b2ed1d51-5c15-4615-8234-d12af7af53f7
retrying ad61f484-6f68-4d9a-8680-2f281309608c as 89195eea-892c-4815-bb0a-71ca8c24b586
retrying cc0800c8-d3a0-4165-a826-1a2f93e9e8ac as a2683ef6-6880-4b51-b67e-648274cfa8b3
retrying fc4c034b-f5eb-453f-9ddb-1a81f991d5db as f856ab01-3446-4063-83c6-60fcd3497274
retrying 2b1e8c7b-5c4f-4f4d-93f5-c27d1526c61a as 67643642-2077-49f9-a807-0123a49ea0a5
retrying 078333b9-9a9c-4b64-9ad9-465d94eb0e88 as a008a5be-e321-4298-8640-2f2d44bda70e
retrying d1ff0f82-df6e-4ee2-885a-a0993d985a2b as 4c73318d-64f1-42d4-827a-c7c04cdd6ef8
retrying 985f5084-f6c6-4451-8ef1-cdebed4ca65a as ac81b85f-c6b1-44d8-af79-e731ef987fe1
retrying c7cd81ef-b83c-4fa4-a872-4b1e31d28c48 as 1bd51a60-f5e4-4f3b-9cdf-309ed254b316
retrying 693355fc-a2ee-4d39-8662-ce1b3b9d85d9 as ec3e7125-6723-4e95-9361-ec1327bbd674
retrying f76e1733-d459-4756-a9a2-17d3184c95cb as 09698d0e-02fa-430f-bb2a-097cd2ab65d5
retrying 8ab4306c-72f4-4cfc-98da-6474860d2cb0 as c74d4

In [ ]:
retry_instances = [next(client.module_instances(tags=["deploy", "retry_0"], path=path)) for path in modules.values()]

In [ ]:
instances = [instance for page in retry_instances for instance in page]
c = Counter([(instance["status"], instance["path"].split("#")[1], instance["target"]) for instance in instances])
print(f"{'Module':<10} | {'Status':<20} | {'Target':<20} | Count")
print("-" * 50)
for (module, status, target), count in c.items():
    print(f"{module:<10} | {status:<20} | {target:<20} | {count}")

Module     | Status               | Target               | Count
--------------------------------------------------
COMPLETED  | hermes_binding       | NIX_SSH_GPU          | 2
COMPLETED  | hermes_binding       | NIX_SSH              | 3
QUEUED     | gmx_frame_select_pdb | NIX                  | 1
QUEUED     | gmx_frame_select_pdb | GADI                 | 2
COMPLETED  | gmx_frame_select_pdb | NIX                  | 3
COMPLETED  | gmx_frame_select_pdb | GADI                 | 2
QUEUED     | gmx_frame_select     | NIX                  | 1
QUEUED     | gmx_frame_select     | GADI                 | 2
COMPLETED  | gmx_frame_select     | NIX                  | 3
COMPLETED  | gmx_frame_select     | GADI                 | 2
RUNNING    | gmx_mmpbsa_tengu     | NIX_SSH_3            | 1
RUNNING    | gmx_mmpbsa_tengu     | GADI                 | 1
COMPLETED  | gmx_mmpbsa_tengu     | NIX_SSH_3            | 2
QUEUED     | gmx_mmpbsa_tengu     | GADI                 | 3
FAILED     | gmx_mmpbsa_tengu 